In [ ]:
from plots.loader import load_timings, load_algorithm
import pandas as pd
import numpy as np
from pathlib import Path

paths = {
    # "naive": Path("./runs/rad_n"),
    # "balanced": Path("./runs/rad_b"),
    # "balanced_s": Path("./runs/rad_bs"),

    # STOKES EXPERIMENTS OLD
    # "stokes": Path("./measurements/stokes-32-32"),
    # "stokes_b": Path("./measurements/stokes-32-32-balanced"),
    
    # EIGER RUNS PARTITIONS 
    # "diel_n": Path("./measurements/partitioning/4_32_drop_at_once_parallel_dielFilterV2real_n"),
    # "diel_b": Path("./measurements/partitioning/4_32_drop_at_once_parallel_dielFilterV2real_b"),
    # "diel_bs": Path("./measurements/partitioning/4_32_drop_at_once_parallel_dielFilterV2real_bs"),
    # "diel_n": Path("./measurements/partitioning/4_32_drop_parallel_dielFilterV2real_n"),
    # "diel_b": Path("./measurements/partitioning/4_32_drop_parallel_dielFilterV2real_b"),
    # "diel_bs": Path("./measurements/partitioning/4_32_drop_parallel_dielFilterV2real_bs"),
    "stokes_n": Path("./measurements/partitioning/4_32_drop_parallel_stokes_n"),
    "stokes_b": Path("./measurements/partitioning/4_32_drop_parallel_stokes_b"),
    "stokes_bs": Path("./measurements/partitioning/4_32_drop_parallel_stokes_bs"),

    # EIGER RUNS 64 MPI 
    # "stokes": Path("./measurements/runs_eiger_16/16_64_drop_parallel_stokes"),
    # "hv15r": Path("./measurements/runs_eiger_16/16_64_drop_parallel_HV15R"),
    # "nlpkkt200": Path("./measurements/runs_eiger_16/16_64_drop_parallel_nlpkkt200"),
    # "nlpkkt240": Path("./measurements/runs_eiger_16/16_64_drop_parallel_nlpkkt240"),
    # "diel": Path("./measurements/runs_eiger_16/16_64_drop_parallel_dielFilterV2real"),

    # DAINT RUNS 64 MPI
    # "diel_old": Path("./measurements/dielFilterV2real/compare_comb/2024-12-15-20:45:04-64-64"),
    # "hv15r_old": Path("./measurements/scalability/HV15R/2024-12-16-10:04:35-64-64"),
    # "nlpkkt200_old": Path("./measurements/scalability/nlpkkt200/2024-12-16-13:27:56-64-64"),
    # "nlpkkt240_old": Path("./measurements/scalability/nlpkkt240/2024-12-16-18:52:01-64-64"),

    # EIGER RUNS 256 MPI 
    # "stokes": Path("./measurements/runs_eiger_64/64_256_drop_parallel_stokes"),
    # "hv15r": Path("./measurements/runs_eiger_64/64_256_drop_parallel_HV15R"),
    # "nlpkkt200": Path("./measurements/runs_eiger_64/64_256_drop_parallel_nlpkkt200"),
    # "nlpkkt240": Path("./measurements/runs_eiger_64/64_256_drop_parallel_nlpkkt240"),
    # "diel": Path("./measurements/runs_eiger_64/64_256_drop_parallel_dielFilterV2real"),
 
    # DAINT RUNS 256 MPI
    # "stokes_old": Path("./measurements/scalability/stokes/2024-12-17-23:43:54-256-256"),
    # "hv15r_old": Path("./measurements/scalability/HV15R/2024-12-16-10:04:40-256-256"),
    # "nlpkkt200_old": Path("./measurements/scalability/nlpkkt200/2024-12-16-13:27:59-256-256"),
    # "nlpkkt240_old": Path("./measurements/scalability/nlpkkt240/2024-12-17-21:59:55-256-256"),
    # "nlpkkt240s_old": Path("./measurements/scalability/nlpkkt240-shuffled/2024-12-15-17:02:34-256-256"),
    
    # EIGER RUNS 256 MPI naive
    # "stokes_naive": Path("./measurements/runs_eiger_64_naive/64_256_drop_parallel_stokes"),
    # "hv15r_naive": Path("./measurements/runs_eiger_64_naive/64_256_drop_parallel_HV15R"),
    # "nlpkkt200_naive": Path("./measurements/runs_eiger_64_naive/64_256_drop_parallel_nlpkkt200"),
    # "nlpkkt240_naive": Path("./measurements/runs_eiger_64_naive/64_256_drop_parallel_nlpkkt240"),
    # "diel_naive": Path("./measurements/runs_eiger_64_naive/64_256_drop_parallel_dielFilterV2real"),    
}
bps = { key: load_timings(path) for key, path in paths.items() }
nodes = len(list(bps.values())[0])
print("NODES:", nodes)

for key, bp in bps.items():
    # assert load_algorithm(paths[key]) == 'drop_parallel'
    assert len(bp) == nodes

In [ ]:
import matplotlib.pyplot as plt

# SCALING_FACTOR = 10**9 # for seconds
SCALING_FACTOR = 10**6
plt.rcParams['font.family'] = 'Computer Modern Roman'
plt.rcParams['text.usetex'] = True

plt.rcParams['axes.labelsize'] = 18
plt.rcParams['xtick.labelsize'] = 18
plt.rcParams['ytick.labelsize'] = 18
STYLES = {
    # color, marker, offst, name
    "comm": 'tab:blue',
    "mult": 'tab:orange',
    "other": 'tab:green',
    "wait": 'tab:grey'
}

In [ ]:
len(bps)

In [ ]:
fig, axs = plt.subplots(1, len(bps), sharey=True, figsize=(1+2*len(bps), 5), dpi=100)

i = 0
print(f"{'Name':<15} {'Total(ms)':>10} {'MB':>10} {'MB/s':>10} {'Comm(ms)':>10} {'Comm(%)':>10} {'Mult(ms)':>10} {'Mult(%)':>10} {'Other(%)':>10} {'(Filter+Deser)/Mult(%)':>25}")
for name, bp in bps.items():
    ax = axs[i]
    m = max([df[df['func'] == 'gemm']['duration'].mean() for df in bp.values()])/SCALING_FACTOR

    timings = []
    mean_f = 0
    mean_d = 0
    mean_m = 0
    mean_o = 0
    mean_w = 0
    mean_b = 0
    for key, df in bp.items():
        wait_s = df[df['func'] == 'wait_all']['duration']
        wait = (wait_s.mean()*(nodes-1))/SCALING_FACTOR #if drop_parallel
        mult_s = df[df['func'] == 'mult']['duration']
        mult = (mult_s.mean() * nodes)/SCALING_FACTOR
        gemm_s = df[df['func'] == 'gemm']['duration']
        gemm = gemm_s.mean()/SCALING_FACTOR
        filter_s = df[df['func'] == 'filter']['duration']
        filter = (filter_s.mean()*(nodes-1))/SCALING_FACTOR
        deserialize_s = df[df['func'] == 'deserialize']['duration']
        deserialize = (deserialize_s.mean()*(nodes-1))/SCALING_FACTOR

        # print(key, wait_s.min()/SCALING_FACTOR, wait_s.mean()/SCALING_FACTOR, wait_s.max()/SCALING_FACTOR)
        mean_f += filter
        mean_d += deserialize 
        mean_m += mult
        mean_w += wait
        mean_o += gemm - mult - wait
        mean_b += df[df['func'] == 'bytes']['bytes'].mean()
        timings.append({
            "key": key,
            "comm": wait,
            "mult": mult,
            "other": gemm - mult - wait,
            "wait": m - gemm,
        })
    mean_o = mean_o/len(bp.items())
    mean_m = mean_m/len(bp.items())
    mean_w = mean_w/len(bp.items())
    mean_f = mean_f/len(bp.items())
    mean_d = mean_d/len(bp.items())
    mean_b = mean_b/len(bp.items()) * 10**-6
    print(f"{name:<15} {m:>10.4f} {mean_b:>10.2f} {mean_b/(mean_w*10**-3):>10.2f} {mean_w:>10.2f} {(mean_w/m)*100:>10.2f} {mean_m:>10.2f} {(mean_m/m)*100:>10.2f} {(mean_o/m)*100:>10.2f} {((mean_f+mean_d)/m)*100:>25.2f}")
    timings = pd.DataFrame(timings)

    keys = timings["key"]
    bottom = np.zeros(len(timings))
    for key, pretty_key in (('comm', 'Communication'), ('mult', 'Multiplication'), ('other', 'Other'), ('wait', 'Wait')):
        ax.bar(keys, timings[key], 1.0, label=pretty_key, bottom=bottom, color=STYLES[key], antialiased=False, rasterized=True, edgecolor="none", linewidth=0)
        bottom += timings[key]

    ax.set_xlabel("MPI Rank", fontsize=12)
    if i == 0:
        ax.set_ylabel('Time [ms]',
                rotation='horizontal',
                loc='top')
        ax.yaxis.set_label_coords(0.35, 1.01) 
    
    ax.set_title(name)

    ax.tick_params(axis='both', direction='in', which='major', pad=5)
    xs = [0,nodes-1]
    ax.get_xaxis().set_ticks(xs, labels=xs)
    i += 1

plt.legend()

In [ ]:
fig.savefig("sections.pdf", bbox_inches="tight")